# Setup

In [ ]:
!pip install pkbar

In [ ]:
from google.colab import drive
from google.colab import files
import sys
import time

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/PSGD Paper/'
results_dir = base_dir + 'RNN/results_xor/'
dpsgd_dir = base_dir + 'DPSGD/'
sys.path.append(base_dir)
sys.path.append(dpsgd_dir)



Mounted at /content/gdrive/


In [ ]:
from densekron import dense_kron
from scankron import scan_kron
from rahkron import rah_kron
from scawkron import scaw_kron
from Shampoo import Shampoo
import preconditioned_stochastic_gradient_descent as psgd

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.autograd import grad
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import time
import math

import pkbar
from tqdm import tqdm
from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px
from torchsummary import summary
import torch.nn as nn
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.get_device_name(0)


'Tesla T4'

# Functions

In [ ]:
def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):

    fig = go.Figure()
    i = 0
    if(xaxis != None):
        for opt in opts:
            fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1
    else:
        for opt in opts:
            fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1

    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
    fig.show()
    fig.write_html(results_dir + title + ".html")

def plot_acc_metrics(xaxis,yaxis,title, x_label,y_label):
 
    fig = go.Figure()
    i = 0
    if(xaxis != None):
        for opt in opts:
            fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1
    else:
        for opt in opts:
            fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1

    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis=dict(range=[0.97, 1]))
    fig.show()
    fig.write_html(results_dir + title + ".html")


def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    if r > 3/4:
        lambd = lambd*omega
    elif r < 1/4:
        lambd = lambd / omega
    return lambd
    

In [ ]:
np.random.seed(0)

# Parameter Settings
BATCH_SIZE = 100
test_BATCH_SIZE = 100
EPOCHS = 50
GAP = 100

# Data Download

In [ ]:
from sklearn.model_selection import train_test_split
batch_size, seq_len0 = 100, 30
dim_in, dim_hidden, dim_out = 2, 20, 1

# generate training data for the add problem
def get_dataset(batch_size=50000):
    seq_len = round(seq_len0 + 0.1*np.random.rand()*seq_len0)
    x = np.zeros([batch_size, seq_len, dim_in])
    y = np.zeros([batch_size, dim_out])
    for i in range(batch_size):
        x[i,:,0] = np.random.choice([-1.0, 1.0], seq_len)

        i1 = int(np.floor(np.random.rand()*0.1*seq_len))
        i2 = int(np.floor(np.random.rand()*0.4*seq_len + 0.1*seq_len))             
        x[i, i1, 1] = 1.0
        x[i, i2, 1] = 1.0
        if x[i,i1,0] == x[i,i2,0]:
            y[i] = -1.0 # lable 0
        else:
            y[i] = 1.0  # lable 1
            
    # tranpose x to dimensions: sequence_length * batch_size * dimension_input 
#     return torch.transpose(x, 1,0).to(device), y.to(device)
    return x, y

X, y = get_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train),torch.Tensor(y_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers=4)

test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test),torch.Tensor(y_test))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = test_BATCH_SIZE, shuffle = True, num_workers=4)

In [ ]:
n_batches = len(train_loader)
n_test_batches = len(test_loader)

In [ ]:
n_test_batches, n_batches

(50, 450)

# Model

In [ ]:
# generate a random orthogonal matrix for recurrent matrix initialization 

# initialize the RNN weights
def initialize_weights():
    Wi_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden+1, dim_hidden])
    Wf_np = np.concatenate([np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden, dim_hidden]),
                                    np.ones([1, dim_hidden])], 0)
    Wo_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden+1, dim_hidden])
    Wc_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden+1, dim_hidden])
    # matrix for the output layer (not for LSTM cell)
    W2_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_hidden+1, dim_out])

    
    Wi = torch.tensor(Wi_np, dtype=torch.float, requires_grad=True).to(device)
    Wf = torch.tensor(Wf_np, dtype=torch.float, requires_grad=True).to(device)
    Wo = torch.tensor(Wo_np, dtype=torch.float, requires_grad=True).to(device)
    Wc = torch.tensor(Wc_np, dtype=torch.float, requires_grad=True).to(device)
    W2 = torch.tensor(W2_np, dtype=torch.float, requires_grad=True).to(device)
    Ws = [Wi, Wf, Wo, Wc, W2]
    return Ws

def get_rand_orth( dim ):
    temp = np.random.normal(size=[dim, dim])
    q, _ = np.linalg.qr(temp)
    return q

ones = torch.ones(batch_size, 1).to(device)


def model(x):
    Wi, Wf, Wo, Wc, W2 = Ws

    def lstm_cell(i, o, state):
        # i: input
        # o: output
        # state: cell state
        input_gate = torch.sigmoid(torch.matmul(torch.cat([i, o, state, ones], 1), Wi))
        forget_gate = torch.sigmoid(torch.matmul(torch.cat([i, o, state, ones], 1), Wf))
        update = torch.matmul(torch.cat([i, o, state, ones], 1), Wc)
        state = forget_gate*state + input_gate*torch.tanh(update)
        
        output_gate = torch.sigmoid(torch.matmul(torch.cat([i, o, state, ones], 1), Wo))
        o = output_gate*torch.tanh(state)
        return o, state
    
    state = torch.zeros([batch_size, dim_hidden]).to(device)
    out = torch.zeros([batch_size, dim_hidden]).to(device)

    for xt in x.transpose(1,0):
        out, state = lstm_cell(xt, out, state)
    y = torch.matmul(torch.cat((out, ones), 1), W2)
    return y

# Loss Function

In [ ]:
def train_loss(data, target):
    out = model(data)
    loss = F.mse_loss(out, target)
    return loss

def test_loss():
    loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            out = model(data)
            loss += F.mse_loss(out, target)
            
    return loss.item()/n_test_batches


def save_start_condition(trainlosslist, testlosslist, timelist):
    trainloss = 0.0

    for (data, target) in tqdm(train_loader, ncols = 80):
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        trainloss += loss

    timelist.append(0)

    testloss = test_loss()

    trainlosslist.append(trainloss.item()/n_batches)
    testlosslist.append(testloss)
    print('Epoch: {}; train loss: {}; test loss: {},  time: {}'.format(0, trainlosslist[-1], testlosslist[-1], np.sum(timelist)))

# SGD

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.numel() for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
    save_start_condition(TrainLoss, TestLoss, times)
for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for (data, target) in train_loader:
        
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        trainloss += loss
        grads = grad(loss, Ws)  
        

        with torch.no_grad():
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for (W,pG) in zip(Ws, grads):
                W -= step_adjust*step_size*pG
            kbar.update(n, values=[("loss", loss.item())])    
            n = n + 1
            
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    
    
#     step_size = 0.01**(1/9)*step_size
    testloss = test_loss()
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
#     print('Epoch: {}; train loss: {}; test loss: {}, train_acc: {}, test_acc:{}, time: {}'\
#      .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'SGD.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

# Adam

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
step_size = 0.001
m0 = [torch.zeros(W.shape).to(device) for W in Ws]
v0 = [torch.zeros(W.shape).to(device) for W in Ws]
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
    save_start_condition(TrainLoss, TestLoss, times)
for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for (data, target) in train_loader:
        
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        trainloss += loss
        grads = grad(loss, Ws, create_graph=True)    

        with torch.no_grad():
            lmbd = min(n/(n+1), 0.9)
            m0 = [lmbd*old + (1.0-lmbd)*new for (old, new) in zip(m0, grads)]
            lmbd = min(n/(n+1), 0.999)
            v0 = [lmbd*old + (1.0-lmbd)*new*new for (old, new) in zip(v0, grads)]
            for (W,m,v) in zip(Ws, m0, v0):
                W -= step_size*(m/torch.sqrt(v + 1e-8))
            kbar.update(n, values=[("loss", loss.item())])    
            n = n + 1
            
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    
    
#     step_size = 0.01**(1/9)*step_size
    testloss = test_loss()
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
#     print('Epoch: {}; train loss: {}; test loss: {}, train_acc: {}, test_acc:{}, time: {}'\
#      .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]<ipython-input-63-86efd3826686>:25: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

100%|████████████████████████████████████████| 450/450 [00:03<00:00, 140.66it/s]


Epoch: 0; train loss: 0.17782674153645833; test loss: 0.1778373146057129,  time: 0
Epoch: 1/50
450/450 [==============================] - 7s 16ms/step - loss: 0.1591 - val_loss: 0.1529
Epoch: 2/50
450/450 [==============================] - 7s 16ms/step - loss: 0.1220 - val_loss: 0.0722
Epoch: 3/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0466 - val_loss: 0.0284
Epoch: 4/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0247 - val_loss: 0.0165
Epoch: 5/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0196 - val_loss: 0.0165
Epoch: 6/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0152 - val_loss: 0.0116
Epoch: 7/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0119 - val_loss: 0.0146
Epoch: 8/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0146 - val_loss: 0.0135
Epoch: 9/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0146 - val_loss: 0.0085

# PSGD

In [ ]:
def training(Ws, epochs, step_size, grad_norm_clip_thr, lambd, omega, dpsgd, update_after, use_damping, file_name):

    Qs = [[0.1*torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
    TrainLoss, TestLoss = [], []
    times = []
    with torch.no_grad():
        save_start_condition(TrainLoss, TestLoss, times)
    n = 0
    for epoch in range(epochs):
        n = 0
        kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
        trainloss = 0.0
        trainacc = 0.0
        t0 = time.time()
        for (data, target) in train_loader:

            data, target = data.to(device), target.to(device)
            loss = train_loss(data, target)
            
            grads = grad(loss, Ws, create_graph=True)
            trainloss += loss
            v = [torch.randn(W.shape).to(device) for W in Ws]
            Hv =  grad(grads, Ws, v)
            
            with torch.no_grad():
                Qs = [dpsgd.update_preconditioner(q, dX, dG, step = 0.01) for (q, dX, dG) in zip(Qs, v, Hv)]
                pre_grads = [dpsgd.precondition_grads(q, g, lambd) for (q,g) in zip(Qs, grads)]
                grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
                step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

                for (W,pG) in zip(Ws, pre_grads):
                    W -= step_adjust*step_size*pG
                    
                if n % update_after == 0 and use_damping and lambd > 1e-10:
                    M = min([0.5*torch.dot(g.reshape(-1,), step_size*pg.reshape(-1,)) for (g, pg) in zip(grads, pre_grads)])
                    loss2 = F.mse_loss(model(data), target)
                    loss1 = loss
                    lambd = update_lambda(loss1, loss2, M,  lambd, omega)
                    
            kbar.update(n, values=[("loss", loss.item())])    
            n = n + 1

        t1 = time.time() - t0
        times.append(t1)
        TrainLoss.append(trainloss.item()/n_batches)

        testloss = test_loss()
        kbar.add(1, values=[("val_loss", testloss)])
        
        TestLoss.append(testloss)
#         step_size = 0.01**(1/9)*step_size
#         print('Epoch: {}; train loss: {}; test loss: {}, time: {}'\
#          .format(epoch+1, TrainLoss[-1], TestLoss[-1], np.sum(times)))

    scipy.io.savemat(results_dir + file_name, {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

## Dense Kron 

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
use_damping = False
kron = dense_kron(Ws, use_4D = False, use_1D = False, use_damping = use_damping, lambd = 150, eta = 1e-5)

grad_norm_clip_thr = 0.1*sum(W.numel() for W in Ws)**0.5
lambd = 150
omega = 0.5

training(Ws, 
         epochs = EPOCHS, 
         step_size = 0.1, 
         grad_norm_clip_thr = grad_norm_clip_thr,
         lambd = lambd,
         omega = omega, 
         dpsgd = kron,
         update_after = 1,
         use_damping = use_damping, 
         file_name = "kron.mat")

In [ ]:
0.1*sum(W.numel() for W in Ws)**0.5

2.193171219946131

## DAMPED DENSE KRON

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
use_damping = True
kron = dense_kron(Ws, use_4D = False, use_1D = False, use_damping = use_damping, lambd = 150, eta = 1e-5)

grad_norm_clip_thr = 0.1*sum(W.numel() for W in Ws)**0.5
lambd = 1
update_after = 5
omega = (19/20)**update_after

training(Ws, 
         epochs = EPOCHS, 
         step_size = 0.1, 
         grad_norm_clip_thr = grad_norm_clip_thr,
         lambd = lambd,
         omega = omega, 
         dpsgd = kron,
         update_after = update_after,
         use_damping = use_damping, 
         file_name = "kron_damped.mat")

  0%|                                                   | 0/450 [00:00<?, ?it/s]<ipython-input-63-86efd3826686>:25: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

100%|████████████████████████████████████████| 450/450 [00:03<00:00, 141.33it/s]


Epoch: 0; train loss: 0.16908454047309027; test loss: 0.16887657165527345,  time: 0
Epoch: 1/50
450/450 [==============================] - 16s 36ms/step - loss: 0.1551 - val_loss: 0.1412
Epoch: 2/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0239 - val_loss: 0.0074
Epoch: 3/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0059 - val_loss: 0.0049
Epoch: 4/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch: 5/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch: 6/50
450/450 [==============================] - 16s 34ms/step - loss: 0.0028 - val_loss: 0.0025
Epoch: 7/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch: 8/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch: 9/50
450/450 [==============================] - 16s 35ms/step - loss: 0.0020 - val_lo

# Shampoo

In [ ]:
from Shampoo import Shampoo
torch.manual_seed(1)
Ws = initialize_weights()
shampoo = Shampoo(Ws, use_1D = False, use_damping = False, epsilon = 0.1)
Qs = [shampoo.initialize_preconditioner(W) for W in Ws]
grad_norm_clip_thr = 0.1*sum(W.numel() for W in Ws)**0.5
step_size = 0.1
TrainLoss, TestLoss = [], []
times = []
with torch.no_grad():
    save_start_condition(TrainLoss, TestLoss, times)
n = 0
for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for (data, target) in train_loader:

        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)


        grads = grad(loss,Ws)

        trainloss += loss
        
        with torch.no_grad():
            Qs = [shampoo.update_preconditioner(q, g) for (q, g) in zip(Qs, grads)]
            pre_grads = [shampoo.precondition_grads(q, g) for (q,g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for (W,pG) in zip(Ws, pre_grads):
                W -= step_adjust*step_size*pG


        kbar.update(n, values=[("loss", loss.item())])    
        n = n + 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)

    testloss = test_loss()
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
#     step_size = 0.01**(1/9)*step_size
#         print('Epoch: {}; train loss: {}; test loss: {}, train_acc: {}, test_acc:{}, time: {}'\
#          .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + "Shampoo.mat", {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]<ipython-input-63-86efd3826686>:25: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

100%|████████████████████████████████████████| 450/450 [00:03<00:00, 140.90it/s]


Epoch: 0; train loss: 0.17273756239149304; test loss: 0.17235149383544923,  time: 0
Epoch: 1/50
450/450 [==============================] - 7s 16ms/step - loss: 0.1566 - val_loss: 0.1545
Epoch: 2/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0809 - val_loss: 0.0268
Epoch: 3/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0234 - val_loss: 0.0164
Epoch: 4/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0160 - val_loss: 0.0184
Epoch: 5/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0131 - val_loss: 0.0102
Epoch: 6/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0111 - val_loss: 0.0094
Epoch: 7/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0096 - val_loss: 0.0076
Epoch: 8/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0088 - val_loss: 0.0068
Epoch: 9/50
450/450 [==============================] - 7s 16ms/step - loss: 0.0077 - val_loss: 0.006

# Comparison

In [ ]:
# opts = ['adam','Kron','SCAN','SCAW', 'fisher_kron','fisher_SCAN','fisher_SCAW','Kron_test','Kron_test2','Kron_damped','fisher_kron_damped','KFAC', 'shampoo']
opts = ['SGD','adam','kron','kron_damped', 'Shampoo']

total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	

In [ ]:
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF', '#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']
# colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [ ]:
for opt in opts:
    # print(opt)
    data = opts_data[opt]
    times[opt] = data.get('Time')
    train_times[opt] = np.cumsum(times[opt])
    test_times[opt] = np.cumsum(times[opt])
    total_train_time[opt] = np.sum(times[opt])
    train_losses[opt] = data.get('TrainLoss').reshape(EPOCHS+1,)
    test_losses[opt] = data.get('TestLoss').reshape(EPOCHS+1,)


In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')